In [1]:
from z3 import *
import itertools

import time
import os

from sat_utils import plot_result, return_coods, optimize

os.makedirs('outputs', exist_ok=True)

In [2]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in itertools.combinations(bool_vars, 2)]

In [3]:
def base_model_order(s,rects,px,py,under,left,W,H):
  #non overlapping condition

  #domain

  #order encoding

  #meaning for left and under

  return s

In [ ]:
def solve(rects,W,opt=False):
  pass

In [ ]:
# import instances
pass
# solve each instance
pass
# plot the results
pass